In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/multi_turn_rewrite/chinese/main')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
%tensorflow_version 1.x

In [3]:
import tensorflow as tf
import time
import numpy as np

from pathlib import Path

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 1.15.0
GPU Enabled: False


In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip('\n')
      word2idx[line] = i
  return word2idx

def align_pad(li):
  max_len = max([len(sent) for sent in li])
  for sent in li:
    if len(sent) < max_len:
      sent += [0] * (max_len - len(sent))

In [0]:
def parse_fn(text):
  h1, h2, q = text.rstrip().split('|')
  char2idx_fn = lambda x: [params['char2idx'].get(c, len(params['char2idx'])) for c in list(x)]
  h1, h2, q = char2idx_fn(h1), char2idx_fn(h2), char2idx_fn(q)
  align_pad([h1, h2])
  return {'query': np.reshape(q, (1, len(q))),
          'history':np.reshape([h1, h2], (1, 2, len(h1)))
  }

In [6]:
params = {'export_dir': '../model/baseline_lstm_export'}

params['char2idx'] = get_vocab('../vocab/char.txt')
params['idx2char'] = {idx: char for char, idx in params['char2idx'].items()}

subdirs = [x for x in Path(params['export_dir']).iterdir()
           if x.is_dir() and 'temp' not in str(x)]
latest = str(sorted(subdirs)[-1])

predict_fn = tf.contrib.predictor.from_saved_model(latest)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from ../model/baseline_lstm_export/1575507415/variables/variables


In [0]:
while True:
  text_inp = input('Input:')
  t0 = time.time()
  predictions = predict_fn(parse_fn(text_inp))
  #print('Output Shape:', predictions['output'].shape)
  print('A:', ' '.join([params['idx2char'].get(idx, '<unk>') for idx in predictions['output'][0] if (idx != 0 and idx != 2)]))
  print('Cost %.2f sec' % (time.time() - t0))
  print()

Input:转行当程序员吧|程序员你有同感吗|我就是
A: 我 就 是 程 序 员
Cost 0.51 sec

Input:现在得癌症的人越来越多了|如何远离癌症|一般家庭承担不起啊
A: 癌 症 一 般 家 庭 承 担 不 起 啊
Cost 0.12 sec

Input:十一种孤独作者是谁|理查德耶茨对吧|这本书讲的啥
A: 德 耶 茨 讲 的 啥
Cost 0.08 sec

Input:你知道秦始皇吗|秦始皇叫嬴政啊|他统一了六国
A: 秦 始 皇 统 一 了 秦 始 皇 统 一 了 六 国 皇
Cost 0.22 sec

Input:对配音演员不了解|配音演员不是统称声优吗|不知道
A: 不 知 道 配 音 演 员
Cost 0.14 sec

Input:权志龙你认识吗|我看到了他的心演的全是他和她的电影|他的歌
A: 权 志 龙 的 歌
Cost 0.06 sec

Input:爱|我爱巧克力|我不爱
A: 我 不 爱 巧 克 力
Cost 0.09 sec

Input:我喜欢猎鹰|那你看看吧|哈哈哈看了
A: 哈 哈 哈 看 我 喜 欢 猎 鹰 了
Cost 0.10 sec

Input:前两天又看了一遍泰坦尼克号给我哭的稀里哗啦的|别看了|你看过类似的特别感人的电影吗
A: 你 看 过 泰 坦 尼 克 号 类 似 的 特 别 感 人 的 电 影 吗
Cost 0.19 sec

Input:你知道人间失格吗|看了这样的小说我唯一想说的是这个男人没救了没救了没救了|他的作者是谁
A: 人 间 失 格 的 作 者 是 谁
Cost 0.10 sec

Input:微软总部在哪|是在美国华盛顿州雷德蒙德市|介绍一下
A: 介 绍 一 下 微 软 总 部
Cost 0.11 sec

Input:你知道围城吗|看着门里门外的人进进出出 自己却不小心就这样坐在门里看了一辈子|他的作者是谁
A: 围 城 的 作 者 是 谁
Cost 0.11 sec

Input:镇魂好看吗|里面有我喜欢的朱一龙嘻嘻|哦回去可以看看
A: 哦 回 去 可 以 看 看 镇 魂
Cost 0.10 sec

Input:讲解一下金瓶梅|看过|什么感觉
A: 什 么 感 觉 金 瓶 梅
Cost 0.09 sec

Input:跑完步会全身都舒坦啊|听说人类很喜欢在跑步的时